#### Langchain, openai key and HUggingFace API key

In [1]:
from langchain_openai import ChatOpenAI
from constant import openai_key
from huggingface_Api import HUGGINGFACEHUB_API_TOKEN

#### Call the openai env key globally

In [2]:
import os
os.environ["OPENAI_API_KEY"] = openai_key

#### From Langchain Initialize the OpenAI LLM

In [3]:
llm = ChatOpenAI(temperature=0.8) # More the value towards 1 more random the LLM would be

##### Testing

In [4]:
text = " what is the capital of india"
print(llm.predict(text)) # also we can use (llm.invoke)

/home/abhisek/Documents/Careers/Studies/Data Science/Projects/Learning-Generative-AI/LangChain/langenv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `predict` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


The capital of India is New Delhi.


#### From Huggingface 
##### We can use opensource model from Huggingface
##### Declare the env globally

In [5]:
os.environ["HUGGINGFACEHUB_API_TOKEN"] = HUGGINGFACEHUB_API_TOKEN

#### We use "google/flan-t5-large" it is a opensource model from Huggingface, also we are testing it below

In [6]:
from langchain import HuggingFaceHub
llm_huggingface=HuggingFaceHub(repo_id="google/flan-t5-large")#,model_kwargs={"temperature":0,"max_lenght":64})

/home/abhisek/Documents/Careers/Studies/Data Science/Projects/Learning-Generative-AI/LangChain/langenv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.huggingface_hub.HuggingFaceHub` was deprecated in langchain-community 0.0.21 and will be removed in 0.2.0. Use HuggingFaceEndpoint instead.
  warn_deprecated(
/home/abhisek/Documents/Careers/Studies/Data Science/Projects/Learning-Generative-AI/LangChain/langenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
text_hug="what is the weather in new delhi and kolkata"
print(llm_huggingface.predict(text_hug))

/home/abhisek/Documents/Careers/Studies/Data Science/Projects/Learning-Generative-AI/LangChain/langenv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `predict` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


The average temperature in New Delhi is 1°C, while in Kolkata it is 


#### compare it with OpenAI LLM

In [8]:
text_openai_llm="what is the weather in new delhi and kolkata"
print(llm.predict(text_openai_llm))

As of now, the weather in New Delhi is 31°C with clear skies and in Kolkata, it is 29°C with partly cloudy skies.


#### Prompt Template

In [21]:
from langchain.prompts import PromptTemplate

prompt_template=PromptTemplate(input_variables=["country"],
               template="Tell me something about the country {country}")

prompt_template.format(country="India")

'Tell me something about the country India'

In [22]:
## To run the prompt and the llm along with it we need llm chain.

from langchain.chains import LLMChain

chain = LLMChain (llm=llm, prompt=prompt_template) ## Open AI LLM

print(chain.run("India"))

/home/abhisek/Documents/Careers/Studies/Data Science/Projects/Learning-Generative-AI/LangChain/langenv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


India is the seventh-largest country in the world by land area and the second-most populous country, with over 1.3 billion people. It is located in South Asia and is known for its diverse culture, rich history, and stunning landscapes. India is home to several major religions, including Hinduism, Buddhism, Islam, and Sikhism. It is also known for its delicious cuisine, vibrant festivals, and Bollywood film industry. The country's economy is one of the fastest-growing in the world, with a strong emphasis on technology and agriculture. India is also known for its ancient architectural wonders, such as the Taj Mahal and the temples of Khajuraho.
